************

********

COSE dataset

Start

In [1]:
from datasets import load_dataset

# Load the CoS-E dataset
dataset = load_dataset('cos_e', 'v1.11')




In [2]:
dataset['train']

Dataset({
    features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation'],
    num_rows: 9741
})

In [3]:
# Display an example
example = dataset['train']
print(f"Question: {example['question'][0]}")
print(example['choices'][0])
print(f"Answer: {example['answer'][0]}")
print(f"Explanation: {example['abstractive_explanation'][0]}")

Question: "There are 10 apples on an apple tree.  Three fall off.  Now there are X apples."  What is this an example of?
['park', 'coloring book', 'garden center', 'math problem', 'gravity']
Answer: math problem
Explanation: webmath is designed to help you solve


In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
# Load the CoS-E dataset
dataset = load_dataset('cos_e', 'v1.11')

Using the latest cached version of the dataset since cos_e couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'v1.11' at /opt/huggingface/datasets/cos_e/v1.11/0.0.0/f01396594b768e0026841176bf770afe58b82fba (last modified on Tue Jul 16 10:04:43 2024).


In [3]:
# Initialize the model and tokenizer
model_name = "t5-base"  # Use "t5-base" or "t5-large" if resources allow
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [4]:
examples=dataset['validation']

In [5]:
for q, choices in zip(examples['question'], examples['choices']):
    print(q)
    print(choices)
    break

A beaver is know for building prowess, their supplies come from where?
['british columbia', 'body of water', 'wooded area', 'pay debts', 'zoo']


In [6]:
x=["question: " + q + " answer: " + " ".join(choices) for q, choices in zip(examples['question'], examples['choices'])]

In [7]:
examples['answer'][0]

'wooded area'

In [8]:
examples['abstractive_explanation'][0]

'Wooden area is only the place for the beaver supplies.'

In [9]:
x[0]

'question: A beaver is know for building prowess, their supplies come from where? answer: british columbia body of water wooded area pay debts zoo'

In [10]:
"answer: " + examples['answer'][0] + " explanation: " + examples['abstractive_explanation'][0]

'answer: wooded area explanation: Wooden area is only the place for the beaver supplies.'

In [11]:
    y = ["answer: " + answer + " explanation: " + explanation for answer, explanation in zip(examples['answer'], examples['abstractive_explanation'])]

In [12]:
y[0]

'answer: wooded area explanation: Wooden area is only the place for the beaver supplies.'

In [13]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = ["question: " + q + " answer: " + " ".join(choices) for q, choices in zip(examples['question'], examples['choices'])]
    targets = ["answer: " + answer + " explanation: " + explanation for answer, explanation in zip(examples['answer'], examples['abstractive_explanation'])]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=256, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs






In [14]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [15]:
encoded_dataset['validation']

Dataset({
    features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1221
})

In [16]:
encoded_dataset['validation']['labels'][0]

[1525,
 10,
 1679,
 15,
 26,
 616,
 7295,
 10,
 2985,
 35,
 616,
 19,
 163,
 8,
 286,
 21,
 8,
 36,
 9,
 624,
 4471,
 5,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [17]:
tokenizer.batch_decode(encoded_dataset['validation']['labels'][0], skip_special_tokens=True)

2024-07-16 12:03:26.200816: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-16 12:03:26.214548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 12:03:26.230434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 12:03:26.234880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 12:03:26.247597: I tensorflow/core/platform/cpu_feature_guar

['answer',
 ':',
 'wood',
 'e',
 'd',
 'area',
 'explanation',
 ':',
 'Wood',
 'en',
 'area',
 'is',
 'only',
 'the',
 'place',
 'for',
 'the',
 'be',
 'a',
 'ver',
 'supplies',
 '.',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>

In [18]:
from torch.utils.data import DataLoader

# Convert dataset to PyTorch tensors
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Create DataLoaders
train_loader = DataLoader(encoded_dataset['train'], batch_size=16, shuffle=True)
val_loader = DataLoader(encoded_dataset['validation'], batch_size=16)


In [22]:
# # Create DataLoaders
# train_dataset = encoded_dataset['train']
# val_dataset = encoded_dataset['validation']
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=4)

In [22]:
# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/home/manavi/myenv/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
# Training loop
def train_loop(model, loader, optimizer, accumulation_steps=2):
    model.train()
    batch_losses = []
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc='Training:')):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss / accumulation_steps  # normalize loss

        batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
        loss.backward()

        if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
            optimizer.step()
            optimizer.zero_grad()  # reset gradients

        batch_losses.append(batch_loss_value)

    # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    loss_value = sum(batch_losses) / len(batch_losses)
    return {'train_loss': loss_value}

# # Validation loop
# def validate_loop(model, loader):
#     model.eval()
#     batch_losses = []

#     with torch.no_grad():
#         for batch in tqdm(loader, desc='Validation:'):
#             inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
#             loss = outputs.loss

#             batch_losses.append(loss.item())

#     loss_value = sum(batch_losses) / len(batch_losses)
#     return {'val_loss': loss_value}




***************

In [25]:
# def validate_loop(model, loader):
#     model.eval()
#     batch_losses = []

#     with torch.no_grad():
#         for batch in tqdm(loader, desc='Validation:'):
#             inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
#             loss = outputs.loss

#             # Generate predictions
#             predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

#             # Decode predictions
#             decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#             decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#             # Print decoded predictions and labels for debugging
#             print(f"Decoded predictions: {decoded_preds}")
#             print(f"Decoded labels: {decoded_labels}")

#             batch_losses.append(loss.item())

#     loss_value = sum(batch_losses) / len(batch_losses)
#     return {'val_loss': loss_value}


***************

just add accuracy validation

In [26]:
# import numpy as np
# from tqdm import tqdm
# from datasets import load_metric

# # Load metrics
# accuracy_metric = load_metric("accuracy")

# def validate_loop(model, loader):
#     model.eval()
#     batch_losses = []
#     accuracy_preds = []
#     accuracy_labels = []

#     with torch.no_grad():
#         for batch in tqdm(loader, desc='Validation:'):
#             inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
#             loss = outputs.loss

#             # Generate predictions
#             predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

#             # Decode predictions and labels
#             decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#             decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#             # Print decoded predictions and labels for debugging
#             print(f"Decoded predictions: {decoded_preds}")
#             print(f"Decoded labels: {decoded_labels}")

#             # Extract the answers from decoded predictions and labels
#             extracted_preds = [pred.split('answer: ')[1].split(' ')[0] for pred in decoded_preds if 'answer: ' in pred]
#             extracted_labels = [label.split('answer: ')[1].split(' ')[0] for label in decoded_labels if 'answer: ' in label]
#             print(f"extracted_preds: {extracted_preds}")
#             print(f"extracted_labels:  {extracted_labels}")
#             # Ensure lengths match for accuracy calculation
#             if len(extracted_preds) == len(extracted_labels):
#                 accuracy_preds.extend(extracted_preds)
#                 accuracy_labels.extend(extracted_labels)

#             batch_losses.append(loss.item())

#     # Calculate accuracy
#     accuracy = accuracy_metric.compute(predictions=accuracy_preds, references=accuracy_labels)
#     loss_value = sum(batch_losses) / len(batch_losses)

#     return {'val_loss': loss_value, 'accuracy': accuracy['accuracy']}


that's correct

add accuracy version 2(customized acc metric)

In [27]:
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

            # Decode predictions and labels
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Print decoded predictions and labels for debugging
            # print("Decoded predictions:")
            # for pred in decoded_preds:
            #     print(f"'{pred}'")
            # print("Decoded labels:")
            # for label in decoded_labels:
            #     print(f"'{label}'")

            # # Extract the answers from decoded predictions and labels
            # extracted_preds = [pred.strip().split('.')[0] for pred in decoded_preds if pred.strip()]
            # extracted_labels = [label.strip().split('.')[0] for label in decoded_labels if label.strip()]
            # Extract the answers from decoded predictions and labels
            extracted_preds = [pred.split('answer: ')[1].split(' ')[0] for pred in decoded_preds if 'answer: ' in pred]
            extracted_labels = [label.split('answer: ')[1].split(' ')[0] for label in decoded_labels if 'answer: ' in label]
            # Ensure lengths match for accuracy calculation
            if len(extracted_preds) == len(extracted_labels):
                accuracy_preds.extend(extracted_preds)
                accuracy_labels.extend(extracted_labels)

            batch_losses.append(loss.item())

    # Calculate accuracy
    correct = sum(p == l for p, l in zip(accuracy_preds, accuracy_labels))
    accuracy = correct / len(accuracy_preds) if accuracy_preds else 0.0
    loss_value = sum(batch_losses) / len(batch_losses)

    return {'val_loss': loss_value, 'accuracy': accuracy}

# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_metrics = train_loop(model, train_loader, optimizer)
    val_metrics = validate_loop(model, val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_metrics['train_loss']:.4f}")
    print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
    print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")



Training::   0%|          | 0/609 [00:00<?, ?it/s]

Validation:: 100%|██████████| 77/77 [02:07<00:00,  1.65s/it]


Epoch 1/3
Train Loss: 0.7436
Validation Loss: 0.1424
Validation Accuracy: 0.2817


Validation:: 100%|██████████| 77/77 [01:22<00:00,  1.07s/it]


Epoch 2/3
Train Loss: 0.1396
Validation Loss: 0.1274
Validation Accuracy: 0.3446


Validation:: 100%|██████████| 77/77 [01:18<00:00,  1.03s/it]

Epoch 3/3
Train Loss: 0.1264
Validation Loss: 0.1218
Validation Accuracy: 0.3604


add bert score for explanation

In [ ]:
# import bert_score
# from datasets import load_metric

# # Load metrics
# accuracy_metric = load_metric("accuracy")

# # Training loop
# def train_loop(model, loader, optimizer, accumulation_steps=16):
#     model.train()
#     batch_losses = []
#     optimizer.zero_grad()

#     for i, batch in enumerate(tqdm(loader, desc='Training:')):
#         inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
#         loss = outputs.loss / accumulation_steps  # normalize loss

#         batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
#         loss.backward()

#         if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
#             optimizer.step()
#             optimizer.zero_grad()  # reset gradients

#         batch_losses.append(batch_loss_value)

#     # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
#     if len(loader) % accumulation_steps != 0:
#         optimizer.step()
#         optimizer.zero_grad()

#     loss_value = sum(batch_losses) / len(batch_losses)
#     return {'train_loss': loss_value}

# # Validation loop
# def validate_loop(model, loader):
#     model.eval()
#     batch_losses = []
#     accuracy_preds = []
#     accuracy_labels = []
#     explanations_preds = []
#     explanations_labels = []

#     with torch.no_grad():
#         for batch in tqdm(loader, desc='Validation:'):
#             inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
#             loss = outputs.loss

#             # Generate predictions
#             predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

#             # Decode predictions and labels
#             decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#             decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#             # Print decoded predictions and labels for debugging
#             print("Decoded predictions:")
#             for pred in decoded_preds:
#                 print(f"'{pred}'")
#             print("Decoded labels:")
#             for label in decoded_labels:
#                 print(f"'{label}'")

#             # Extract the answers from decoded predictions and labels
#             # extracted_preds = [pred.strip().split('.')[0] for pred in decoded_preds if pred.strip()]
#             # extracted_labels = [label.strip().split('.')[0] for label in decoded_labels if label.strip()]

#             extracted_preds = [pred.split('answer: ')[1].split(' ')[0] for pred in decoded_preds if 'answer: ' in pred]
#             extracted_labels = [label.split('answer: ')[1].split(' ')[0] for label in decoded_labels if 'answer: ' in label]
            
#             # Ensure lengths match for accuracy calculation
#             if len(extracted_preds) == len(extracted_labels):
#                 accuracy_preds.extend(extracted_preds)
#                 accuracy_labels.extend(extracted_labels)

#             # Collect explanations for BERTScore
            
#             # explanations_preds.extend(decoded_preds)
#             # explanations_labels.extend(decoded_labels)
#             explanations_preds = [pred.split('explanation: ')[1].split(' ')[0] for pred in decoded_preds if 'explanation: ' in pred]
#             explanations_labels = [label.split('explanation: ')[1].split(' ')[0] for label in decoded_labels if 'explanation: ' in label]

#             batch_losses.append(loss.item())

#     # Calculate accuracy
#     correct = sum(p == l for p, l in zip(accuracy_preds, accuracy_labels))
#     accuracy = correct / len(accuracy_preds) if accuracy_preds else 0.0
#     loss_value = sum(batch_losses) / len(batch_losses)

#     # Calculate BERTScore
#     P, R, F1 = bert_score.score(explanations_preds, explanations_labels, lang="en", rescale_with_baseline=True)
#     bertscore_avg = F1.mean().item()

#     return {'val_loss': loss_value, 'accuracy': accuracy, 'bertscore': bertscore_avg}

# # Training and validation
# num_epochs = 3
# for epoch in range(num_epochs):
#     train_metrics = train_loop(model, train_loader, optimizer)
#     val_metrics = validate_loop(model, val_loader)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Train Loss: {train_metrics['train_loss']:.4f}")
#     print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
#     print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
#     print(f"Validation BERTScore: {val_metrics['bertscore']:.4f}")




above code has problem,decode predictions has empty

In [1]:

# # Validation loop with BERTScore for explanations and predictions
# def validate_loop(model, loader):
#     model.eval()
#     batch_losses = []
#     accuracy_preds = []
#     accuracy_labels = []
#     explanations_preds = []
#     explanations_labels = []

#     with torch.no_grad():
#         for batch in tqdm(loader, desc='Validation:'):
#             inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
#             loss = outputs.loss

#             # Generate predictions
#             predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

#             # Decode predictions and labels
#             decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#             decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#             # Print decoded predictions and labels for debugging
#             print("Decoded predictions:")
#             for pred in decoded_preds:
#                 print(f"'{pred}'")
#             print("Decoded labels:")
#             for label in decoded_labels:
#                 print(f"'{label}'")

#             # Extract the answers from decoded predictions and labels
#             extracted_preds = [pred.split('answer: ')[1].split(' ')[0] for pred in decoded_preds if 'answer: ' in pred]
#             extracted_labels = [label.split('answer: ')[1].split(' ')[0] for label in decoded_labels if 'answer: ' in label]

#             # Ensure lengths match for accuracy calculation
#             if len(extracted_preds) == len(extracted_labels):
#                 accuracy_preds.extend(extracted_preds)
#                 accuracy_labels.extend(extracted_labels)

#             # Collect explanations for BERTScore
#             # explanations_preds.extend(decoded_preds)
#             # explanations_labels.extend(decoded_labels)
#             explanations_preds = [pred.split('explanation: ')[1].split(' ')[0] for pred in decoded_preds if 'explanation: ' in pred]
#             explanations_labels = [label.split('explanation: ')[1].split(' ')[0] for label in decoded_labels if 'explanation: ' in label]

#             batch_losses.append(loss.item())

#     # Calculate BERTScore for answers (accuracy_preds and accuracy_labels)
#     P_ans, R_ans, F1_ans = bert_score.score(accuracy_preds, accuracy_labels, lang="en", rescale_with_baseline=True)
#     bertscore_ans_avg = F1_ans.mean().item()

#     # Calculate BERTScore for explanations (explanations_preds and explanations_labels)
#     P_exp, R_exp, F1_exp = bert_score.score(explanations_preds, explanations_labels, lang="en", rescale_with_baseline=True)
#     bertscore_exp_avg = F1_exp.mean().item()

#     loss_value = sum(batch_losses) / len(batch_losses)

#     return {'val_loss': loss_value, 'bertscore_ans': bertscore_ans_avg, 'bertscore_exp': bertscore_exp_avg}

# # Training and validation
# num_epochs = 3
# for epoch in range(num_epochs):
#     train_metrics = train_loop(model, train_loader, optimizer)
#     val_metrics = validate_loop(model, val_loader)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Train Loss: {train_metrics['train_loss']:.4f}")
#     print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
#     print(f"Validation BERTScore (Answers): {val_metrics['bertscore_ans']:.4f}")
#     print(f"Validation BERTScore (Explanations): {val_metrics['bertscore_exp']:.4f}")


توی پردیکشن ها جمله خالی هم هست.

In [20]:
def convert_to_sentences(list_of_lists):
    sentences = [' '.join(inner_list) for inner_list in list_of_lists]
    return sentences

# Example usage
list_of_lists = [
    ["The", "capital", "of", "France", "is", "Paris."],
    ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog."],
    ["Artificial", "intelligence", "is", "transforming", "the", "world."]
]

sentences = convert_to_sentences(list_of_lists)
print(sentences)


['The capital of France is Paris.', 'The quick brown fox jumps over the lazy dog.', 'Artificial intelligence is transforming the world.']


In [ ]:
import bert_score
from datasets import load_metric
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

            # Decode predictions and labels
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Print decoded predictions and labels for debugging
            # print("Decoded predictions:")
            # for pred in decoded_preds:
            #     print(f"'{pred}'")
            # print("Decoded labels:")
            # for label in decoded_labels:
            #     print(f"'{label}'")

            # # Extract the answers from decoded predictions and labels
            # extracted_preds = [pred.strip().split('.')[0] for pred in decoded_preds if pred.strip()]
            # extracted_labels = [label.strip().split('.')[0] for label in decoded_labels if label.strip()]
            # Extract the answers from decoded predictions and labels
            extracted_preds = [pred.split('answer: ')[1].split(' ')[0] for pred in decoded_preds if 'answer: ' in pred]
            extracted_labels = [label.split('answer: ')[1].split(' ')[0] for label in decoded_labels if 'answer: ' in label]
            
            explanations_preds = [pred.split('explanation: ')[1].split(' ') for pred in decoded_preds if 'explanation: ' in pred]
            explanations_labels = [label.split('explanation: ')[1].split(' ') for label in decoded_labels if 'explanation: ' in label]
            print(convert_to_sentences(explanations_preds))
            print(convert_to_sentences(explanations_labels))
            
            # Ensure lengths match for accuracy calculation
            if len(extracted_preds) == len(extracted_labels):
                accuracy_preds.extend(extracted_preds)
                accuracy_labels.extend(extracted_labels)

            batch_losses.append(loss.item())

    # Calculate accuracy
    correct = sum(p == l for p, l in zip(accuracy_preds, accuracy_labels))
    accuracy = correct / len(accuracy_preds) if accuracy_preds else 0.0
    
    
    # Calculate BERTScore for explanations (explanations_preds and explanations_labels)
    P_exp, R_exp, F1_exp,_= bert_score.score(convert_to_sentences(explanations_preds), convert_to_sentences(explanations_labels), lang="en", rescale_with_baseline=True)
    bertscore_exp_avg = F1_exp.mean().item()

    loss_value = sum(batch_losses) / len(batch_losses)
    # return {'val_loss': loss_value, 'accuracy': accuracy}
    # return {'val_loss': loss_value, 'bertscore_exp': bertscore_exp_avg}
    return {'val_loss': loss_value, 'accuracy': accuracy, 'bertscore_exp': bertscore_exp_avg}


# # Training and validation
# num_epochs = 1
# for epoch in range(num_epochs):
#     train_metrics = train_loop(model, train_loader, optimizer)
#     val_metrics = validate_loop(model, val_loader)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Train Loss: {train_metrics['train_loss']:.4f}")
#     print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
#     print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    
# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_metrics = train_loop(model, train_loader, optimizer)
    val_metrics = validate_loop(model, val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_metrics['train_loss']:.4f}")
    print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
    print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    print(f"Validation BERTScore (Explanations): {val_metrics['bertscore_exp']:.4f}")



End

In [30]:
# def convert_to_sentences(list_of_lists):
#     sentences = [' '.join(inner_list) for inner_list in list_of_lists]
#     return sentences

# # Example usage
# list_of_lists = [
#     ["The", "capital", "of", "France", "is", "Paris."],
#     ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog."],
#     ["Artificial", "intelligence", "is", "transforming", "the", "world."]
# ]

# sentences = convert_to_sentences(list_of_lists)
# print(sentences)


['The capital of France is Paris.', 'The quick brown fox jumps over the lazy dog.', 'Artificial intelligence is transforming the world.']


phase2 <br>
adding dream to model

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/DREAM") # can take a couple minutes, be patient!
tokenizer = AutoTokenizer.from_pretrained("t5-11b")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Question: "There are 10 apples on an apple tree.  Three fall off.  Now there are X apples."  What is this an example of?
['park', 'coloring book', 'garden center', 'math problem', 'gravity']
Answer: math problem
Explanation: webmath is designed to help you solve

In [11]:
input_string = "$answer$ ; $question$ = [SITUATION] There are 10 apples on an apple tree.  Three fall off.  Now there are X apples. What is this an example of?  math or gravity?. [QUERY] consequence"
input_ids = tokenizer.encode(input_string, return_tensors="pt").to(device)
output = model.generate(input_ids, max_length=50)
tokenizer.batch_decode(output, skip_special_tokens=True)

['$answer$ = [immoral_consequence] The cat has a hard time balancing its weight with the apples it is carrying.']

In [9]:
input_string = "$answer$ ; $question$ = [SITUATION] There are 10 apples on an apple tree.  Three fall off.  Now there are X apples. What is this an example of? math or gravity?.  [QUERY] rot"
input_ids = tokenizer.encode(input_string, return_tensors="pt").to(device)
output = model.generate(input_ids, max_length=50)
tokenizer.batch_decode(output, skip_special_tokens=True)
# ["$answer$ = It's wrong to damage other people's property."]

['$answer$ = It is good to know the math behind things.']

kari ke mikhaim bokonim ine ke tarighe soal dadan avaz konim ya kolan javabaro nadim.

create elaboration from dream model in two ways

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation'],
        num_rows: 1221
    })
})

In [9]:
print(dataset['validation']['question'][0])
print(dataset['validation']['choices'][0])

A beaver is know for building prowess, their supplies come from where?
['british columbia', 'body of water', 'wooded area', 'pay debts', 'zoo']


with or of choices

In [16]:
def format_question(example):
    situation = example['question']
    choices = " or ".join(example['choices'])
    input_string = f"$answer$ ; $question$ = [SITUATION] {situation}  {choices} [QUERY] rot"
    example['formatted_question'] = input_string
    return example

# Apply the formatting function to the dataset
formatted_dataset = dataset.map(format_question)

# Print a few examples to verify the formatting
print(formatted_dataset['train'][0]['formatted_question'])
print(formatted_dataset['validation'][0]['formatted_question'])


$answer$ ; $question$ = [SITUATION] "There are 10 apples on an apple tree.  Three fall off.  Now there are X apples."  What is this an example of?  park or coloring book or garden center or math problem or gravity [QUERY] rot
$answer$ ; $question$ = [SITUATION] A beaver is know for building prowess, their supplies come from where?  british columbia or body of water or wooded area or pay debts or zoo [QUERY] rot


In [27]:
# input_string = "$answer$ ; $question$ = [SITUATION] There are 10 apples on an apple tree.  Three fall off.  Now there are X apples. What is this an example of? math or gravity?.  [QUERY] rot"

rot dream

In [17]:
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'formatted_question'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'formatted_question'],
        num_rows: 1221
    })
})

apply dream to formatted_question

In [7]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=16384, bias=False)
              (k): Linear(in_features=1024, out_features=16384, bias=False)
              (v): Linear(in_features=1024, out_features=16384, bias=False)
              (o): Linear(in_features=16384, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 128)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=65536, bias=False)
              (wo): Linear(in_features=65536, out_features=1024, bias=False)
         

In [22]:
def generate_output(example):
    input_string = example['formatted_question']
    input_ids = tokenizer.encode(input_string, return_tensors="pt").to(device)
    output = model.generate(input_ids, max_length=100)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    example['generated_output'] = generated_text
    return example


In [29]:
formatted_dataset['validation'][0]

{'id': '080ef6941410139d6869e78122bc741e',
 'question': 'A beaver is know for building prowess, their supplies come from where?',
 'choices': ['british columbia',
  'body of water',
  'wooded area',
  'pay debts',
  'zoo'],
 'answer': 'wooded area',
 'abstractive_explanation': 'Wooden area is only the place for the beaver supplies.',
 'extractive_explanation': 'their supplies come',
 'formatted_question': '$answer$ ; $question$ = [SITUATION] A beaver is know for building prowess, their supplies come from where?  british columbia or body of water or wooded area or pay debts or zoo [QUERY] rot'}

In [36]:
example=formatted_dataset['validation'][2]
input_string = example['formatted_question']
input_ids = tokenizer.encode(input_string, return_tensors="pt").to(device)
output = model.generate(input_ids, max_length=100)
print(output)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
example['generated_output'] = generated_text
print(example['generated_output'])

tensor([[    0,  1514,  3247,  3321,  3229,  3274,    71,   861,    31,     7,
           577,    19,   952,    12,    36, 13006,     5,     1]],
       device='cuda:0')
$answer$ = A child's play is likely to be boring.


In [32]:
example['formatted_question']

'$answer$ ; $question$ = [SITUATION] A beaver is know for building prowess, their supplies come from where?  british columbia or body of water or wooded area or pay debts or zoo [QUERY] rot'

In [37]:
# Apply the function to generate outputs
result_dataset = formatted_dataset.map(generate_output)

# Print a few examples to verify the outputs
print(result_dataset['train'][0]['generated_output'])
print(result_dataset['validation'][0]['generated_output'])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

$answer$ = It is good to be aware of your surroundings.
$answer$ =


In [38]:
# Example to save to a directory named "formatted_dataset"
result_dataset.save_to_disk('./formatted_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/9741 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1221 [00:00<?, ? examples/s]

**********************

add dream model text to input of t5 and fine-tuning

**************************

******

####################################################

************

****************

### debugging not important

**debugging phase and bleurt score that doesn't work because of google api not work**

def validation loop using bert score for explanation score

In [22]:
# %pip install datasets transformers torch tqdm bert-score bleurt


In [21]:
from datasets import load_dataset, load_metric

In [51]:
# !pip install --upgrade pip  # ensures that pip is current
# !git clone https://github.com/google-research/bleurt.git
# !pip install ./bleurt

In [30]:
!unzip /home/manavi/bleurt-base-128.zip

Archive:  /home/manavi/bleurt-base-128.zip
   creating: bleurt-base-128/
  inflating: bleurt-base-128/vocab.txt  
  inflating: bleurt-base-128/bert_config.json  
   creating: bleurt-base-128/variables/
  inflating: bleurt-base-128/variables/variables.index  
  inflating: bleurt-base-128/variables/variables.data-00000-of-00001  
  inflating: bleurt-base-128/bleurt_config.json  
  inflating: bleurt-base-128/saved_model.pb  


In [40]:
# bleurt_metric = load("bleurt", "/home/manavi/bleurt-base-128")

In [42]:
# Load metrics
# accuracy_metric = load_metric("accuracy")
# bert_score_metric = load_metric("bertscore")
# bleurt_metric = load_metric("bleurt")

# accuracy_metric = load_metric("accuracy")
# bert_score_metric = load_metric("bertscore", trust_remote_code=True)
# bleurt_metric = load_metric("bleurt",trust_remote_code=True, checkpoint="bleurt-base-128")

# Load metrics
accuracy_metric = load_metric("accuracy")
bert_score_metric = load_metric("bertscore", trust_remote_code=True)

In [45]:

# Training loop
def train_loop(model, loader, optimizer, accumulation_steps=16):
    model.train()
    batch_losses = []
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc='Training:')):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss / accumulation_steps  # normalize loss

        batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
        loss.backward()

        if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
            optimizer.step()
            optimizer.zero_grad()  # reset gradients

        batch_losses.append(batch_loss_value)

    # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    loss_value = sum(batch_losses) / len(batch_losses)
    return {'train_loss': loss_value}

# Validation loop
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []
    bertscore_preds = []
    bertscore_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Prepare for accuracy metric
            accuracy_preds.extend(decoded_preds)
            accuracy_labels.extend(decoded_labels)

            # Prepare for BERTScore metric
            bertscore_preds.extend(decoded_preds)
            bertscore_labels.extend(decoded_labels)

            batch_losses.append(loss.item())

    # Compute metrics
    loss_value = sum(batch_losses) / len(batch_losses)
    accuracy = accuracy_metric.compute(predictions=accuracy_preds, references=accuracy_labels)
    bert_score = bert_score_metric.compute(predictions=bertscore_preds, references=bertscore_labels, lang="en")
    bert_score_avg = sum(bert_score['f1']) / len(bert_score['f1'])

    return {
        'val_loss': loss_value,
        'accuracy': accuracy['accuracy'],
        'bert_score': bert_score_avg
    }



In [47]:
# Training loop
def train_loop(model, loader, optimizer, accumulation_steps=16):
    model.train()
    batch_losses = []
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc='Training:')):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss / accumulation_steps  # normalize loss

        batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
        loss.backward()

        if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
            optimizer.step()
            optimizer.zero_grad()  # reset gradients

        batch_losses.append(batch_loss_value)

    # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    loss_value = sum(batch_losses) / len(batch_losses)
    return {'train_loss': loss_value}

# Validation loop
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []
    bertscore_preds = []
    bertscore_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Filter out empty strings
            decoded_preds = [pred for pred in decoded_preds if pred.strip()]
            decoded_labels = [label for label in decoded_labels if label.strip()]

            # Prepare for accuracy metric (assuming labels are in the form of text that needs to be converted to integers)
            accuracy_preds.extend([int(pred.split()[1]) for pred in decoded_preds if pred.split()[1].isdigit()])
            accuracy_labels.extend([int(label.split()[1]) for label in decoded_labels if label.split()[1].isdigit()])

            # Prepare for BERTScore metric
            bertscore_preds.extend(decoded_preds)
            bertscore_labels.extend(decoded_labels)

            batch_losses.append(loss.item())

    # Compute metrics
    loss_value = sum(batch_losses) / len(batch_losses)
    accuracy = accuracy_metric.compute(predictions=accuracy_preds, references=accuracy_labels)
    bert_score = bert_score_metric.compute(predictions=bertscore_preds, references=bertscore_labels, lang="en")
    bert_score_avg = sum(bert_score['f1']) / len(bert_score['f1'])

    return {
        'val_loss': loss_value,
        'accuracy': accuracy['accuracy'],
        'bert_score': bert_score_avg
    }

In [26]:
# %pip install git+https://github.com/google-research/bleurt.git

In [33]:
# %pip install git+https://github.com/google-research/bleurt.git'

In [27]:
# %pip install bert_score

In [ ]:
# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_metrics = train_loop(model, train_loader, optimizer)
    val_metrics = validate_loop(model, val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_metrics['train_loss']:.4f}")
    print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
    print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    print(f"Explanation Score: {val_metrics['explanation_score']}")

In [49]:
# Training loop
def train_loop(model, loader, optimizer, accumulation_steps=16):
    model.train()
    batch_losses = []
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc='Training:')):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss / accumulation_steps  # normalize loss

        batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
        loss.backward()

        if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
            optimizer.step()
            optimizer.zero_grad()  # reset gradients

        batch_losses.append(batch_loss_value)

    # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    loss_value = sum(batch_losses) / len(batch_losses)
    return {'train_loss': loss_value}

# Validation loop
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []
    bertscore_preds = []
    bertscore_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Filter out empty strings and check lengths
            decoded_preds = [pred for pred in decoded_preds if pred.strip()]
            decoded_labels = [label for label in decoded_labels if label.strip()]

            # Debugging output
            print(f"Decoded predictions: {decoded_preds}")
            print(f"Decoded labels: {decoded_labels}")

            # Prepare for accuracy metric (assuming labels are in the form of text that needs to be converted to integers)
            accuracy_preds.extend([int(pred.split()[1]) for pred in decoded_preds if pred.split()[1].isdigit()])
            accuracy_labels.extend([int(label.split()[1]) for label in decoded_labels if label.split()[1].isdigit()])

            # Prepare for BERTScore metric
            bertscore_preds.extend(decoded_preds)
            bertscore_labels.extend(decoded_labels)

            batch_losses.append(loss.item())

    # Ensure no empty lists before computing metrics
    if not accuracy_preds or not accuracy_labels:
        raise ValueError("Empty list found for accuracy predictions or labels.")
    if not bertscore_preds or not bertscore_labels:
        raise ValueError("Empty list found for BERTScore predictions or labels.")

    # Compute metrics
    loss_value = sum(batch_losses) / len(batch_losses)
    accuracy = accuracy_metric.compute(predictions=accuracy_preds, references=accuracy_labels)
    bert_score = bert_score_metric.compute(predictions=bertscore_preds, references=bertscore_labels, lang="en")
    bert_score_avg = sum(bert_score['f1']) / len(bert_score['f1'])

    return {
        'val_loss': loss_value,
        'accuracy': accuracy['accuracy'],
        'bert_score': bert_score_avg
    }

In [50]:
# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_metrics = train_loop(model, train_loader, optimizer)
    val_metrics = validate_loop(model, val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_metrics['train_loss']:.4f}")
    print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
    print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    print(f"Explanation Score: {val_metrics['explanation_score']}")

Validation::   3%|▎         | 1/39 [00:06<03:56,  6.22s/it]

Decoded predictions: []
Decoded labels: ['answer: wooded area explanation: Wooden area is only the place for the beaver supplies.', 'answer: go downtown explanation: Students do everything', 'answer: play tag explanation: tag different ways to play tag', 'answer: great outdoors explanation: woods are must', 'answer: club explanation: he wants a club', 'answer: whirlpool bath explanation: Whirlpool bath definition', 'answer: industrial area explanation: Coimbatore Industrial Park', 'answer: martorell explanation: A man can take a seat in martorell', 'answer: house explanation: A house is a building that functions as a home', 'answer: photograph explanation: photograph A patriotic guy with a camera is looking for a bald eagle', 'answer: heartburn explanation: a meat is tasty', 'answer: bushel basket explanation: Bushel - Wikipedia', 'answer: bank explanation: Rivers flow trough valleys.', 'answer: appliance store explanation: it is more comfortable to normal room', 'answer: country expla

Validation::   5%|▌         | 2/39 [00:12<03:49,  6.20s/it]

Decoded predictions: []
Decoded labels: ['answer: common interests explanation: List of hobbies - Wikipedia', 'answer: heartburn explanation: We generally take some chewables to prevent the heartburn.', 'answer: violin case explanation: he needs a violin case', 'answer: fitness center explanation: be FITNESS | Delafield, WI - Fitness Center, Gym & Personal Training', 'answer: butcher shop explanation: goods in specialized stores', 'answer: park explanation: children playing park', "answer: house explanation: People cannot have their convenience in another person's House.", 'answer: small town explanation: a big city is capital of country', 'answer: round explanation: approximately round a round face', 'answer: trash explanation: Waste are unwanted or unusable', 'answer: state park explanation: California State Parks', 'answer: closed room explanation: This word was most relevant', 'answer: lie down explanation: By the waters of Babylon we lay down...', 'answer: desire to travel explana

Validation::   8%|▊         | 3/39 [00:18<03:43,  6.20s/it]

...

Validation::  97%|█████████▋| 38/39 [03:57<00:06,  6.25s/it]

Decoded predictions: []
Decoded labels: ['answer: learn new things explanation: e-learning for kids', 'answer: become disillusioned explanation: Disillusioned definition and meaning', 'answer: unable to work explanation: Inability to Work and Eligibility', 'answer: desire to travel explanation: Here is a treasure trovE', "answer: you're curious explanation: Youthful inexperience", 'answer: mentally challenged explanation: sometime disability is happened for mentally challenged', 'answer: attract attention explanation: attract attention does someone flirt with many people at once', 'answer: learn about explanation: learn more about that', 'answer: many words explanation: young child is cute', 'answer: confusing explanation: Definition of confuse', 'answer: false information explanation: Rivers flow trough valleys.', 'answer: record album explanation: DANCE PERFORMING IN RECORD ALBUM', 'answer: lots of space explanation: This word is Most relavant', 'answer: smell good explanation: smell

Validation:: 100%|██████████| 39/39 [04:01<00:00,  6.19s/it]

Decoded predictions: []
Decoded labels: ['answer: problems explanation: problems lead to wrong doings', 'answer: newspaper explanation: Newspaper is produce everyday.', 'answer: transfer of information explanation: Rivers flow trough valleys.', 'answer: supermarket explanation: obtain new soap to brought on supermarket', "answer: ocean explanation: as the planet's"]


ValueError: Empty list found for accuracy predictions or labels.

In [51]:
# Training loop
def train_loop(model, loader, optimizer, accumulation_steps=16):
    model.train()
    batch_losses = []
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc='Training:')):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss / accumulation_steps  # normalize loss

        batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
        loss.backward()

        if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
            optimizer.step()
            optimizer.zero_grad()  # reset gradients

        batch_losses.append(batch_loss_value)

    # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    loss_value = sum(batch_losses) / len(batch_losses)
    return {'train_loss': loss_value}

# Validation loop
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []
    bertscore_preds = []
    bertscore_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Debugging output
            print(f"Decoded predictions: {decoded_preds}")
            print(f"Decoded labels: {decoded_labels}")

            # Filter out empty strings and check lengths
            decoded_preds = [pred for pred in decoded_preds if pred.strip()]
            decoded_labels = [label for label in decoded_labels if label.strip()]

            # Ensure lengths match for accuracy metric
            if len(decoded_preds) != len(decoded_labels):
                print(f"Length mismatch: {len(decoded_preds)} predictions, {len(decoded_labels)} labels")
                continue

            # Prepare for accuracy metric (assuming labels are in the form of text that needs to be converted to integers)
            try:
                accuracy_preds.extend([int(pred.split()[1]) for pred in decoded_preds if len(pred.split()) > 1 and pred.split()[1].isdigit()])
                accuracy_labels.extend([int(label.split()[1]) for label in decoded_labels if len(label.split()) > 1 and label.split()[1].isdigit()])
            except IndexError as e:
                print(f"Index error: {e}")
                continue

            # Prepare for BERTScore metric
            bertscore_preds.extend(decoded_preds)
            bertscore_labels.extend(decoded_labels)

            batch_losses.append(loss.item())

    # Ensure no empty lists before computing metrics
    if not accuracy_preds or not accuracy_labels:
        print(f"Accuracy preds: {accuracy_preds}")
        print(f"Accuracy labels: {accuracy_labels}")
        raise ValueError("Empty list found for accuracy predictions or labels.")
    if not bertscore_preds or not bertscore_labels:
        print(f"BERTScore preds: {bertscore_preds}")
        print(f"BERTScore labels: {bertscore_labels}")
        raise ValueError("Empty list found for BERTScore predictions or labels.")

    # Compute metrics
    loss_value = sum(batch_losses) / len(batch_losses)
    accuracy = accuracy_metric.compute(predictions=accuracy_preds, references=accuracy_labels)
    bert_score = bert_score_metric.compute(predictions=bertscore_preds, references=bertscore_labels, lang="en")
    bert_score_avg = sum(bert_score['f1']) / len(bert_score['f1'])

    return {
        'val_loss': loss_value,
        'accuracy': accuracy['accuracy'],
        'bert_score': bert_score_avg
    }

In [52]:
# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_metrics = train_loop(model, train_loader, optimizer)
    val_metrics = validate_loop(model, val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_metrics['train_loss']:.4f}")
    print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
    print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    print(f"Validation BERTScore: {val_metrics['bert_score']}")

Validation::   3%|▎         | 1/39 [00:06<03:59,  6.30s/it]

Decoded predictions: ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Decoded labels: ['answer: wooded area explanation: Wooden area is only the place for the beaver supplies.', 'answer: go downtown explanation: Students do everything', 'answer: play tag explanation: tag different ways to play tag', 'answer: great outdoors explanation: woods are must', 'answer: club explanation: he wants a club', 'answer: whirlpool bath explanation: Whirlpool bath definition', 'answer: industrial area explanation: Coimbatore Industrial Park', 'answer: martorell explanation: A man can take a seat in martorell', 'answer: house explanation: A house is a building that functions as a home', 'answer: photograph explanation: photograph A patriotic guy with a camera is looking for a bald eagle', 'answer: heartburn explanation: a meat is tasty', 'answer: bushel basket explanation: Bushel - Wikipedia', 'answer: bank explanation: Rive

Validation::   5%|▌         | 2/39 [00:12<03:53,  6.32s/it]

Decoded predictions: ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Decoded labels: ['answer: common interests explanation: List of hobbies - Wikipedia', 'answer: heartburn explanation: We generally take some chewables to prevent the heartburn.', 'answer: violin case explanation: he needs a violin case', 'answer: fitness center explanation: be FITNESS | Delafield, WI - Fitness Center, Gym & Personal Training', 'answer: butcher shop explanation: goods in specialized stores', 'answer: park explanation: children playing park', "answer: house explanation: People cannot have their convenience in another person's House.", 'answer: small town explanation: a big city is capital of country', 'answer: round explanation: approximately round a round face', 'answer: trash explanation: Waste are unwanted or unusable', 'answer: state park explanation: California State Parks', 'answer: closed room explanation: This word 

Validation::   8%|▊         | 3/39 [00:18<03:47,  6.32s/it]

...

Validation::  97%|█████████▋| 38/39 [04:02<00:06,  6.46s/it]

Decoded predictions: ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Decoded labels: ['answer: learn new things explanation: e-learning for kids', 'answer: become disillusioned explanation: Disillusioned definition and meaning', 'answer: unable to work explanation: Inability to Work and Eligibility', 'answer: desire to travel explanation: Here is a treasure trovE', "answer: you're curious explanation: Youthful inexperience", 'answer: mentally challenged explanation: sometime disability is happened for mentally challenged', 'answer: attract attention explanation: attract attention does someone flirt with many people at once', 'answer: learn about explanation: learn more about that', 'answer: many words explanation: young child is cute', 'answer: confusing explanation: Definition of confuse', 'answer: false information explanation: Rivers flow trough valleys.', 'answer: record album explanation: DANCE PERFOR

Validation:: 100%|██████████| 39/39 [04:07<00:00,  6.34s/it]

Decoded predictions: ['', '', '', '', '']
Decoded labels: ['answer: problems explanation: problems lead to wrong doings', 'answer: newspaper explanation: Newspaper is produce everyday.', 'answer: transfer of information explanation: Rivers flow trough valleys.', 'answer: supermarket explanation: obtain new soap to brought on supermarket', "answer: ocean explanation: as the planet's"]
Length mismatch: 0 predictions, 5 labels
Accuracy preds: []
Accuracy labels: []


ValueError: Empty list found for accuracy predictions or labels.

***************

debugging